## SQL Agent Configuration ##

1. Are basic alerts for severity 16 and higher setup and enabled to the correct operator, covering all databases? 
  * If not, recommend running Add Error Event Alerts.sql to enable alert notification. 
  * Consider disabling the severity 20 error alert because of common nuisance emails.
2. Are the following jobs enabled on the server and completing successfully? 
3. Check Job History on the following jobs to confirm all are succeeding: 
  * Automated Index Maintenance 
  ** Look for any type of similarly-named index maintenance job to ensure index maintenance is being completed routinely. 
  ** Actually check the log table to verify activity and no errors. This or similar: select * from dbaadmin.dbo.indexmaintlog order by id desc 
  * Volume Stats Insert 
  ** Do a select on the table, this should get a few months of data back. Any big jumps or trends? SELECT TOP (1000) * FROM [DBAHOUND].[DBO].[VOLUMESTATS] ORDER BY DATETIMEPERFORMED DESC 
  * Job Failure Notification 
  ** Can also be named “Add Job Failure Notification”
4. Change the SQL Server Agent log history retention. 
  * Check how far back history goes for important jobs, such as backup jobs. Make sure to change the total row count from default 1000 rows to 10000 or larger, especially on servers that have replication jobs running or multiple large-output jobs, to prevent loss of historical Information. To store hourly transaction log backups rows for 3 months (recommended), you would also need to increase the max rows per job to at least 2160!

In [ ]:
SELECT TOP (1000) * FROM [DBALOGGING].[DBO].[VOLUMESTATS] ORDER BY DATETIMEPERFORMED DESC

Drives

1. Record SQL file locations 
*  Run “select * from sys.master_files” 
** Are there any SQL server files that have been added to the C: drive? If so, they should be moved.

In [ ]:
select * from sys.master_files

In [ ]:
--Volume Stats.sql
PRINT('Volume Stats Output Window')
select 
  volume_letter = UPPER(vs.volume_mount_point)
, volume_name = vs.logical_volume_name
, file_system_type
, drive_size_GB = MAX(CONVERT(decimal(19,2), vs.total_bytes/1024./1024./1024. ))
, drive_free_space_GB = MAX(CONVERT(decimal(19,2), vs.available_bytes/1024./1024./1024. ))
, drive_percent_free = MAX(CONVERT(decimal(5,2), vs.available_bytes * 100.0 / vs.total_bytes))
FROM
   sys.master_files AS f CROSS APPLY
   sys.dm_os_volume_stats(f.database_id, f.file_id) vs --only return volumes where there is database file (data or log)
 GROUP BY vs.volume_mount_point, vs.file_system_type, vs.logical_volume_name
 ORDER BY volume_letter 

## Disaster Recovery ##
1.	Review current backup status, look for missing FULL or missing TLOG backups for DB’s not in SIMPLE.
  *	**Backup history.sql**
2.	Retention settings
  *	If local backups are being taken, view the backup folder and verify that retention policy is deleting old backups.
  *	Verify that backup chains are intact - .bak’s should not be deleted before dependent .dif and .trn files.

In [ ]:
--Backup hisotry.sql
--Looks for a complete backup history, displaying the latest backup of each type.

--See also toolbox/multiserver backup history.sql for health checks.
PRINT('Backup History Output Window')
use master
go
--sql2012 and above
select 
	database_Name
	, backuptype 
	, d.recovery_model_desc
	, BackupDate = MAX(BackupDate)
	, d.state_desc
	, d.is_read_only
	, dm.Replica_Role		--SQL 2012+
 from sys.databases d
 inner join 
 (
select distinct 
	database_name
	, database_id = db_id(database_name)
	, backuptype = case type	WHEN 'D' then 'Database'
							WHEN 'I' then 'Differential database'
							WHEN 'L' then 'Transaction Log'
							WHEN 'F' then 'File or filegroup'
							WHEN 'G' then 'Differential file'
							WHEN 'P' then 'Partial'
							WHEN 'Q' then 'Differential partial' END
	, BackupDate	=	MAX(backup_finish_date)  	
	from msdb.dbo.backupset bs							
 group by Database_name, type
 UNION 
 select distinct
	db_name(d.database_id)
	, d.database_id
	, backuptype = 'Database'
	, null
	FROM master.sys.databases d
 UNION
 select distinct
	db_name(d.database_id)
	, d.database_id
	, backuptype = 'Transaction Log'
	, null
  FROM master.sys.databases d
  where d.recovery_model_desc in ('FULL', 'BULK_LOGGED')
 ) a
 on d.database_id = a.database_id
 
 --SQL 2012+
 LEFT OUTER JOIN ( SELECT  database_id 
						 ,	Replica_Role		= CASE WHEN database_state_desc IS NOT NULL and last_received_time is null THEN 'PRIMARY '
															WHEN database_state_desc IS NOT NULL and last_received_time is not null THEN 'SECONDARY' 
															ELSE null END
															from sys.dm_hadr_database_replica_states) dm
						 on dm.database_id = a.database_id
WHERE database_name not in ('model','tempdb')
and not (backuptype = 'transaction log' and recovery_model_desc = 'SIMPLE')
group by database_name, backuptype, d.recovery_model_desc, d.state_desc, d.is_read_only, dm.replica_role
order by backuptype, recovery_model_desc, database_name asc
go

 /*
--for SQL 2000 and above
select distinct 
	  database_name	= d.name 
	, a.backuptype	
	, RecoveryModel	=	databasepropertyex(d.name, 'Recovery')  
	, BackupDate	=	Max(a.backup_finish_date)  
	from master.dbo.sysdatabases d
	left outer join 
	(		select distinct 
			database_name
			, backuptype = case type	WHEN 'D' then 'Database'
									WHEN 'I' then 'Differential database backup'
									WHEN 'L' then 'Transaction Log'
									WHEN 'F' then 'File or filegroup'
									WHEN 'G' then 'Differential file'
									WHEN 'P' then 'Partial'
									WHEN 'Q' then 'Differential partial' END
			, backup_finish_date	=	MAX(backup_finish_date)  	
			from msdb.dbo.backupset bs							
		 group by Database_name, type
		 UNION 
		 select distinct
			  d.name
			, backuptype = 'Database'
			, null
			FROM master.dbo.sysdatabases d
		 UNION
		 select distinct
			  d.name
			, backuptype = 'Transaction Log'
			, null
		  FROM master.dbo.sysdatabases d
		  where databasepropertyex(d.name, 'Recovery') in ('FULL', 'BULK_LOGGED')
  ) a
	on d.name = a.database_name
 group by d.name , backuptype ,	databasepropertyex(d.name, 'Recovery')
order by backuptype, RecoveryModel, BackupDate asc
 */
 
--granular backup history
SELECT TOP 1000
		bs.database_name
	, backuptype = CASE 
							WHEN bs.type = 'D' and bs.is_copy_only = 0 then 'Full Database'
							WHEN bs.type = 'D' and bs.is_copy_only = 1 then 'Full Copy-Only Database'
							WHEN bs.type = 'I' then 'Differential database backup'
							WHEN bs.type = 'L' then 'Transaction Log'
							WHEN bs.type = 'F' then 'File or filegroup'
							WHEN bs.type = 'G' then 'Differential file'
							WHEN bs.type = 'P' then 'Partial'
							WHEN bs.type = 'Q' then 'Differential partial' END + ' Backup'
	, bs.recovery_model
	, BackupStartDate = bs.Backup_Start_Date
	, BackupFinishDate = bs.Backup_Finish_Date
	, LatestBackupLocation = bf.physical_device_name
	, backup_size_mb			=	bs.backup_size / 1024./1024.
	, compressed_backup_size_mb =	bs.compressed_backup_size /1024./1024.
	, database_backup_lsn -- For tlog and differential backups, this is the checkpoint_lsn of the FULL backup it is based on. 
	, checkpoint_lsn
	, begins_log_chain
	FROM msdb.dbo.backupset bs	
	LEFT OUTER JOIN msdb.dbo.[backupmediafamily] bf
	on bs.[media_set_id] = bf.[media_set_id]
	WHERE bs.backup_start_date > dateadd(mo, -1, getdate())
	--and database_name = 'w' --optionally filter by database
	ORDER BY  bs.database_name asc, bs.Backup_Start_Date desc;
 

 
 /*
  --Latest Restore
 select d.name, Latest_Restore = max(restore_date)
	from sys.databases d
	LEFT OUTER JOIN msdb.dbo.restorehistory rh on d.name = rh.destination_database_name
	group by d.name
	order by Latest_Restore desc

*/


--Look for backups to NUL, a sign that someone doesn't know what they're doing to the tlog. (Probably VEEAM. Bad VEEAM.)
--This is bad. Backup to NUL is just truncating the log without backing up the log, breaking the tlog chain. Any subsequent tlog backups are broken until a FULL backup restarts a valid chain.
--Do not allow VEEAM or other VSS-based backup solutions to do backups to NUL. 
--In VEEAM, this is somewhere near the "application aware backups" or similar settings menu in various settings. Disable this. 
SELECT 
	  bs.database_name
	, backuptype = CASE 
							WHEN bs.type = 'D' and bs.is_copy_only = 0 then 'Full Database'
							WHEN bs.type = 'D' and bs.is_copy_only = 1 then 'Full Copy-Only Database'
							WHEN bs.type = 'I' then 'Differential database backup'
							WHEN bs.type = 'L' then 'Transaction Log'
							WHEN bs.type = 'F' then 'File or filegroup'
							WHEN bs.type = 'G' then 'Differential file'
							WHEN bs.type = 'P' then 'Partial'
							WHEN bs.type = 'Q' then 'Differential partial' END + ' Backup'
	, bs.recovery_model
	, BackupStartDate = bs.Backup_Start_Date
	, BackupFinishDate = bs.Backup_Finish_Date
	, LatestBackupLocation = bf.physical_device_name
	, backup_size_mb			=	bs.backup_size / 1024./1024.
	, compressed_backup_size_mb =	bs.compressed_backup_size /1024./1024.
	, database_backup_lsn -- For tlog and differential backups, this is the checkpoint_lsn of the FULL backup it is based on. 
	, checkpoint_lsn
	, begins_log_chain
	FROM msdb.dbo.backupset bs	
	LEFT OUTER JOIN msdb.dbo.[backupmediafamily] bf
	on bs.[media_set_id] = bf.[media_set_id]
	where bf.physical_device_name = 'NUL'
	ORDER BY  bs.database_name asc, bs.Backup_Start_Date desc;

## Database Mail ##
1.	View Database Mail log for failures
  * Look for unsent mails, unstarted broker, stopped process in Database Mail Diag.sql
2.	Verify functionality by sending a test email to managed.sql@sparkhound.com or appropriate distribution group. 

In [ ]:
--Database Mail Diag.sql
PRINT('Databse Mail Diagnostic Output Window')
SELECT is_broker_enabled FROM sys.databases WHERE name = 'msdb' ; -- should be 1
EXECUTE msdb.dbo.sysmail_help_status_sp ; --should say STARTED
--EXECUTE msdb.dbo.sysmail_start_sp --start the database mail queues;
GO

--Find recent unsent emails, hopefully there are none
SELECT m.send_request_date, m.recipients, m.copy_recipients, m.blind_copy_recipients
, m.[subject], sent_account = a.name, m.send_request_user, m.sent_status
, Error_Description = l.description 
FROM msdb.dbo.sysmail_allitems m
LEFT OUTER JOIN msdb.dbo.sysmail_account a
	ON m.sent_account_id = a.account_id
LEFT OUTER JOIN msdb.dbo.sysmail_event_log AS l  
    ON m.mailitem_id = l.mailitem_id  
WHERE	1=1
AND     m.send_request_date > dateadd(day, -45, sysdatetime()) -- Only show recent day(s)
AND		m.sent_status <> 'sent' -- Possible values are sent (successful), unsent (in process), retrying (failed but retrying), failed (no longer retrying)
ORDER BY m.send_request_date DESC;
GO

--Send mail test
--exec msdb.dbo.sp_send_dbmail @profile_name ='hotmail', @recipients ='williamdassaf@hotmail.com', @subject ='test', @body = 'test'

--ALTER DATABASE msdb SET ENABLE_BROKER;

## Security ##
1.	Run **Public Permissions.sql** to identify any SELECT and/or EXECUTE permissions granted to the public server role.
  *	Do not recommend granting ALTER, CONTROL, REFERENCES, TAKE OWNERSHIP, VIEW DEFINITION, to anyone who is not already an admin. We should be able to revoke this permission from public with no replacement unless the application is doing something with dynamically creating/altering objects. These permissions shouldn’t be needed by anyone not doing deployments of code changes and shouldn’t be necessary for day-to-day operation unless the application is doing something with dynamically creating/altering objects.
  *	Need developer opinions on how DELETEs, INSERTs, UPDATEs are actually made. Then, grant these DELETE, INSERT or UPDATE permissions to something that is clearly not a read-only user, such as an application service user.
  *	It is not uncommon to grant EXECUTE for a whole database to an application or app service user, but it should not be granted to a read-only user. Perhaps there is a subset of sprocs that are for reporting that a read-only user should have access to EXECUTE, would need developer insight on that.
  *	SELECT is the easiest topic. 
  ** There is a SELECT permission granted to public on the entire [dbo] schema. All the individual GRANT SELECT on dbo objects can be revoked and replaced by a single GRANT SELECT ON SCHEMA::[dbo] to one or more application service users including read-only users.
  ** For GRANT SELECT on objects on other non-dbo schemas, these could be used by read-write application service users and read-only service users. We should replace the public permission with GRANT SELECT on these objects to any user that could access these tables.

In [ ]:
--Public Permissions.sql
PRINT('Public Permissions Output Window')
SELECT @@SERVERNAME


--Server Level Security
SELECT rm.state_desc, rm.permission_name, principal_name = QUOTENAME(u.name),  u.type_desc
,  TSQL = rm.state_desc + N' ' + rm.permission_name + 
	CASE WHEN e.name is not null THEN ' ON ENDPOINT::[' + e.name + '] ' ELSE '' END +
	N' TO ' + cast(QUOTENAME(u.name COLLATE DATABASE_DEFAULT) as nvarchar(256)) + ';'
,  TSQL = N'REVOKE ' + rm.permission_name +
	CASE WHEN e.name is not null THEN ' ON ENDPOINT::[' + e.name + '] ' ELSE '' END +
	 N' TO ' + cast(QUOTENAME(u.name COLLATE DATABASE_DEFAULT) as nvarchar(256)) + ';', *
FROM sys.server_permissions rm
inner join sys.server_principals u 
on rm.grantee_principal_id = u.principal_id
left outer join sys.endpoints e
on e.endpoint_id = major_id and class_desc = 'ENDPOINT'
where u.name not like '##%' 
and u.name = 'public'
order by rm.permission_name, u.name


--Database role membership
--Multi Database
declare @TSQL nvarchar(4000) = 'use [?]; 
SELECT DB_NAME();
SELECT DISTINCT	QUOTENAME(r.name) as database_role_name, r.type_desc, QUOTENAME(d.name) as principal_name, d.type_desc
,	Add_TSQL = ''EXEC sp_addrolemember @membername = N'''''' + d.name COLLATE DATABASE_DEFAULT + '''''', @rolename = N'''''' + r.name + ''''''''
,	Drop_TSQL = ''EXEC sp_droprolemember @membername = N'''''' + d.name COLLATE DATABASE_DEFAULT + '''''', @rolename = N'''''' + r.name + ''''''''
FROM	sys.database_role_members rm
inner join sys.database_principals r on rm.role_principal_id = r.principal_id
inner join sys.database_principals d on rm.member_principal_id = d.principal_id
where d.name = ''public''
';
EXEC sp_MSforeachdb @TSQL
GO

--Multi-database database permissions
--script is too long for sp_msforeachdb, had to roll our own.

declare @TSQL varchar(8000) = null, @dbcount int = 0, @x int = 0, @dbname varchar(256) = null
declare @dblist table (id int not null identity(1,1) primary key, dbname varchar(256)  not null )
insert into @dblist (dbname)
select name from sys.databases where name <> 'tempdb' and state_desc = 'ONLINE' 
order by database_id
select @dbcount = count(dbname) from @dblist

while (@x <= @dbcount)
BEGIN
	select @dbname = dbname from @dblist d where @x = d.id;

	select @TSQL = 	'USE [' + @dbname  + '];
	SELECT DB_NAME();
	SELECT	Permission_State_Desc	=	perm.state_desc
		,	Permission_Name			=	perm.permission_name 
		,	Permission_Object_Name	= ISNULL(QUOTENAME(s.name ) + ''.'','''') + QUOTENAME(obj.name COLLATE database_default) + CASE WHEN cl.name COLLATE database_default is null THEN '''' ELSE ''.'' + QUOTENAME(cl.name COLLATE database_default) END 			
		,	Object_Type_Desc		=	obj.type_desc   
		,	Principal_Name			=	QUOTENAME(u.name COLLATE database_default) 
		,	User_Type				=	u.type_desc 
		,	Create_TSQL = perm.state_desc + N'' '' + perm.permission_name 
			+ case when obj.name COLLATE database_default is not null THEN + N'' ON '' + sc.class_desc + ''::'' + ISNULL(QUOTENAME(s.name COLLATE database_default) + ''.'','''') + QUOTENAME(obj.name COLLATE database_default) ELSE '''' END 
			+ CASE WHEN cl.column_id IS NULL THEN '' '' ELSE ''('' + QUOTENAME(cl.name COLLATE database_default) + '')'' END 
			+ N'' TO '' + QUOTENAME(u.name COLLATE database_default)
		,	Revoke_TSQL = N''REVOKE '' + perm.permission_name 
			+ case when obj.name COLLATE database_default is not null THEN + N'' ON '' + sc.class_desc + ''::'' + ISNULL(QUOTENAME(s.name COLLATE database_default) + ''.'','''') + QUOTENAME(obj.name COLLATE database_default) ELSE '''' END 
			+ CASE WHEN cl.column_id IS NULL THEN '' '' ELSE ''('' + QUOTENAME(cl.name COLLATE database_default) + '')'' END 
			+ N'' TO '' + QUOTENAME(u.name COLLATE database_default) 
			, *
	FROM sys.database_permissions AS perm 
	INNER JOIN sys.database_principals AS u	ON perm.grantee_principal_id = u.principal_id
	LEFT OUTER JOIN (--https://msdn.microsoft.com/en-us/library/ms188367.aspx			
						select name, object_id, schema_id, is_ms_shipped, class_desc=''OBJECT'', type_desc from sys.objects 
						union all
						select name, 0, null, null, ''DATABASE'', ''DATABASE''  from sys.databases 	
						union all
						select  name, schema_id, null, null, ''SCHEMA'', ''SCHEMA'' from sys.schemas
						union all
						select name, principal_id, null, null,  ''USER'', type_desc from sys.database_principals where type_desc in (''WINDOWS_USER'',''SQL_USER'',''ASYMMETRIC_KEY_MAPPED_USER'',''CERTIFICATE_MAPPED_USER'', ''WINDOWS_GROUP'',''EXTERNAL_GROUPS'')
						union all
						select name, principal_id, null, null,  ''USER'', type_desc from sys.database_principals where type_desc in (''WINDOWS_USER'',''SQL_USER'',''ASYMMETRIC_KEY_MAPPED_USER'',''CERTIFICATE_MAPPED_USER'', ''WINDOWS_GROUP'',''EXTERNAL_GROUPS'')
						union all
						select name, principal_id, null, null, ''APPLICATION ROLE'', type_desc from sys.database_principals where type_desc in (''APPLICATION_ROLE'')
						union all
						select name, principal_id, null, null, ''ROLE'', type_desc from sys.database_principals where type_desc in (''DATABASE_ROLE'')
						union all
						select name, assembly_id, null, null, ''ASSEMBLY'', ''ASSEMBLY'' from sys.assemblies 
						union all
						select name, user_type_id, null, null, ''TYPE'', ''USER TYPE'' from sys.types 
						union all
						select name, xml_collection_id, null, null, ''XML SCHEMA COLLECTION'', ''XML SCHEMA COLLECTION'' from sys.xml_schema_collections
						union all
						select name COLLATE database_default, message_type_id, null, null, ''MESSAGE TYPE'', ''MESSAGE TYPE'' from sys.service_message_types
						union all
						select name COLLATE database_default, service_contract_id, null, null, ''CONTRACT'', ''CONTRACT'' from sys.service_contracts
						union all
						select name COLLATE database_default, service_id, null, null, ''SERVICE'', ''SERVICE'' from sys.services
						union all
						select name COLLATE database_default, remote_service_binding_id, null, null, ''REMOTE SERVICE BINDING'', ''REMOTE SERVICE BINDING'' from sys.remote_service_bindings
						union all
						select name COLLATE database_default, route_id, null, null, ''ROUTE'', ''ROUTE''  from sys.routes
						union all
						select name COLLATE database_default, fulltext_catalog_id, null, null, ''FULLTEXT CATALOG'', ''FULLTEXT CATALOG''  from sys.fulltext_catalogs
						union all
						select name, symmetric_key_id, null, null, ''SYMMETRIC KEY'', ''SYMMETRIC KEY''  from sys.symmetric_keys
						union all
						select name, certificate_id, null, null, ''CERTIFICATE'', ''CERTIFICATE'' from sys.certificates
						union all
						select name, asymmetric_key_id, null, null, ''ASYMMETRIC KEY'', ''ASYMMETRIC KEY'' from sys.asymmetric_keys
				) obj
	ON perm.major_id = obj.[object_id] 
	INNER JOIN sys.securable_classes sc on sc.class = perm.class 
	and sc.class_desc = obj.class_desc
	LEFT OUTER JOIN sys.schemas s ON s.schema_id = obj.schema_id
	LEFT OUTER JOIN sys.columns cl ON cl.column_id = perm.minor_id AND cl.[object_id] = perm.major_id
	where 1=1
	and u.name = ''public''
	and perm.major_id > 0
	--Ignore internal principals
	and u.name COLLATE database_default not in (''dbo'',''guest'',''INFORMATION_SCHEMA'',''sys'',''MS_DataCollectorInternalUser'',''PolicyAdministratorRole'',''ServerGroupReaderRole''
	,''ServerGroupAdministratorRole'',''TargetServersRole'',''SQLAgentUserRole'',''UtilityCMRReader'',''SQLAgentOperatorRole'',''dc_operator'',''dc_proxy'',''dc_admin'',''db_ssisadmin'',''db_ssisltduser'',''db_ssisoperator''
	,''UtilityIMRWriter'',''UtilityIMRReader'',''RSExecRole'',''DatabaseMailUserRole'')
	--Ignore ## principals
	and u.name COLLATE database_default not like ''##%##''
	--Ignore built-in svc accounts (not recommended anyway!)
	and u.name COLLATE database_default not like ''NT SERVICE%''
	--Ignore MS shipped internal objects 
	and (obj.is_ms_shipped = 0 or obj.is_ms_shipped is null) 
	--Ignore system sprocs (be aware of your naming conventions!)
	--and (obj.name not like ''sp_%'' or obj.name is null)
	--Ignore SSMS Diagramming Objects
	AND (ISNULL(QUOTENAME(s.name ) + ''.'','''') + QUOTENAME(obj.name COLLATE database_default) + CASE WHEN cl.name COLLATE database_default is null THEN '''' ELSE ''.'' + QUOTENAME(cl.name COLLATE database_default) END)
	NOT IN (''[dbo].[fn_diagramobjects]'',
			''[dbo].[sp_helpdiagrams]'',
			''[dbo].[sp_helpdiagramdefinition]'',
			''[dbo].[sp_creatediagram]'',
			''[dbo].[sp_renamediagram]'',
			''[dbo].[sp_alterdiagram]'',
			''[dbo].[sp_dropdiagram]'')
	--Ignore Database Tuning Advisor Objects
	AND (ISNULL(QUOTENAME(s.name ) + ''.'','''') + QUOTENAME(obj.name COLLATE database_default) + CASE WHEN cl.name COLLATE database_default is null THEN '''' ELSE ''.'' + QUOTENAME(cl.name COLLATE database_default) END)
	NOT IN (''[dbo].[dt_generateansiname]'',
			''[dbo].[dt_adduserobject]'',
			''[dbo].[dtproperties]'',
			''[dbo].[dt_setpropertybyid]'',
			''[dbo].[dt_getobjwithprop]'',
			''[dbo].[dt_getpropertiesbyid]'',
			''[dbo].[dt_setpropertybyid_u]'',
			''[dbo].[dt_getobjwithprop_u]'',
			''[dbo].[dt_getpropertiesbyid_u]'',
			''[dbo].[dt_dropuserobjectbyid]'',
			''[dbo].[dt_droppropertiesbyid]'',
			''[dbo].[dt_verstamp006]'',
			''[dbo].[dt_verstamp007]'',
			''[dbo].[dt_getpropertiesbyid_vcs]'',
			''[dbo].[dt_displayoaerror]'',
			''[dbo].[dt_adduserobject_vcs]'',
			''[dbo].[dt_addtosourcecontrol]'',
			''[dbo].[dt_checkinobject]'',
			''[dbo].[dt_checkoutobject]'',
			''[dbo].[dt_isundersourcecontrol]'',
			''[dbo].[dt_removefromsourcecontrol]'',
			''[dbo].[dt_validateloginparams]'',
			''[dbo].[dt_vcsenabled]'',
			''[dbo].[dt_whocheckedout]'',
			''[dbo].[dt_getpropertiesbyid_vcs_u]'',
			''[dbo].[dt_displayoaerror_u]'',
			''[dbo].[dt_addtosourcecontrol_u]'',
			''[dbo].[dt_checkinobject_u]'',
			''[dbo].[dt_checkoutobject_u]'',
			''[dbo].[dt_isundersourcecontrol_u]'',
			''[dbo].[dt_validateloginparams_u]'',
			''[dbo].[dt_whocheckedout_u]'')
	order by Object_Type_Desc, Principal_Name';


	exec (@TSQL);
	select @x = @x + 1;
END